In [31]:
from util import *
import numpy as np

In [32]:
from util import *

class RestrictedBoltzmannMachine():
    '''
    For more details : A Practical Guide to Training Restricted Boltzmann Machines https://www.cs.toronto.edu/~hinton/absps/guideTR.pdf
    '''
    def __init__(self, ndim_visible, ndim_hidden, is_bottom=False, image_size=[28,28], is_top=False, n_labels=10, batch_size=10):

        """
        Args:
          ndim_visible: Number of units in visible layer.
          ndim_hidden: Number of units in hidden layer.
          is_bottom: True only if this rbm is at the bottom of the stack in a deep belief net. Used to interpret visible layer as image data with dimensions "image_size".
          image_size: Image dimension for visible layer.
          is_top: True only if this rbm is at the top of stack in deep beleif net. Used to interpret visible layer as concatenated with "n_label" unit of label data at the end. 
          n_label: Number of label categories.
          batch_size: Size of mini-batch.
        """
       
        self.ndim_visible = ndim_visible

        self.ndim_hidden = ndim_hidden

        self.is_bottom = is_bottom
        if is_bottom : self.image_size = image_size
        
        self.is_top = is_top

        if is_top : self.n_labels = 10

        self.batch_size = batch_size        
                
        self.delta_bias_v = 0

        self.delta_weight_vh = 0

        self.delta_bias_h = 0

        self.bias_v = np.random.normal(loc=0.0, scale=0.01, size=(self.ndim_visible))

        self.weight_vh = np.random.normal(loc=0.0, scale=0.01, size=(self.ndim_visible,self.ndim_hidden))

        self.bias_h = np.random.normal(loc=0.0, scale=0.01, size=(self.ndim_hidden))
        
        self.delta_weight_v_to_h = 0

        self.delta_weight_h_to_v = 0        
        
        self.weight_v_to_h = None
        
        self.weight_h_to_v = None

        self.learning_rate = 0.01
        
        self.momentum = 0.7

        self.print_period = 5000
        
        self.rf = { # receptive-fields. Only applicable when visible layer is input data
            "period" : 5000, # iteration period to visualize
            "grid" : [5,5], # size of the grid
            "ids" : np.random.randint(0,self.ndim_hidden,25) # pick some random hidden units
            }
        
        return

        
    def cd1(self, visible_trainset, n_iterations=10000):
        
        """Contrastive Divergence with k=1 full alternating Gibbs sampling

        Args:
          visible_trainset: training data for this rbm, shape is (size of training set, size of visible layer)
          n_iterations: number of iterations of learning (each iteration learns a mini-batch)
        """

        print ("learning CD1")
        
        n_samples = visible_trainset.shape[0]
        index = 0 

        for it in range(1):
            # Select next mini-batch
            next_index = index + self.batch_size

            if next_index < n_samples:
                v_0 = visible_trainset[index:next_index]
            else:
                v_0 = np.concatenate((visible_trainset[index:],visible_trainset[:next_index-n_samples]))
                print(index)
            index = next_index % n_samples

	        # [Done TASK 4.1] run k=1 alternating Gibbs sampling : v_0 -> h_0 ->  v_1 -> h_1.
            # you may need to use the inference functions 'get_h_given_v' and 'get_v_given_h'.
            # note that inference methods returns both probabilities and activations (samples from probablities) and you may have to decide when to use what.

            print(v_0)

            h_0_prob, h_0_bin = self.get_h_given_v(v_0)
            v_1_prob, v_1_bin = self.get_v_given_h(h_0_prob)
            h_1_prob, h_1_bin = self.get_h_given_v(v_1_bin)
            
            # [Done TASK 4.1] update the parameters using function 'update_params'
            self.update_params(v_0, h_0_bin, v_1_prob, h_1_prob)

            # visualize once in a while when visible layer is input images
            
            if it % self.rf["period"] == 0 and self.is_bottom:
                viz_rf(weights=self.weight_vh[:,self.rf["ids"]].reshape((self.image_size[0],self.image_size[1],-1)), it=it, grid=self.rf["grid"])

            # print progress
            
            if it % self.print_period == 0 :
                print ("iteration=%7d recon_loss=%4.4f"%(it, np.linalg.norm(visible_trainset - visible_trainset)))
        
        return
    

    def update_params(self,v_0,h_0,v_k,h_k):

        """Update the weight and bias parameters.

        You could also add weight decay and momentum for weight updates.

        Args:
           v_0: activities or probabilities of visible layer (data to the rbm)
           h_0: activities or probabilities of hidden layer
           v_k: activities or probabilities of visible layer
           h_k: activities or probabilities of hidden layer
           all args have shape (size of mini-batch, size of respective layer)
        """

        # [DONE TASK 4.1] get the gradients from the arguments (replace the 0s below) and update the weight and bias parameters
        
        self.delta_bias_v = self.learning_rate * np.mean(v_0 - v_k, axis=0)
        self.delta_weight_vh = self.learning_rate * (v_0.T @ h_0 - v_k.T @ h_k) / v_0.shape[0]
        self.delta_bias_h = self.learning_rate * np.mean(h_0 - h_k, axis=0)

        assert self.delta_bias_v.shape[0] == v_0.shape[1]

        self.bias_v += self.delta_bias_v
        self.weight_vh += self.delta_weight_vh
        self.bias_h += self.delta_bias_h
        
        return

    def get_h_given_v(self, visible_minibatch):
        
        """Compute probabilities p(h|v) and activations h ~ p(h|v) 

        Uses undirected weight "weight_vh" and bias "bias_h"
        
        Args: 
           visible_minibatch: shape is (size of mini-batch, size of visible layer)
        Returns:        
           tuple ( p(h|v) , h) 
           both are shaped (size of mini-batch, size of hidden layer)
        """
        
        assert self.weight_vh is not None

        n_samples = visible_minibatch.shape[0]

        # [Done TASK 4.1] compute probabilities and activations (samples from probabilities) of hidden layer (replace the zeros below) 

        probs = sigmoid(visible_minibatch @ self.weight_vh + self.bias_h)
        binary_states = np.random.binomial(1, probs, size=None)
        
        return probs, binary_states


    def get_v_given_h(self,hidden_minibatch):
        
        """Compute probabilities p(v|h) and activations v ~ p(v|h)

        Uses undirected weight "weight_vh" and bias "bias_v"
        
        Args: 
           hidden_minibatch: shape is (size of mini-batch, size of hidden layer)
        Returns:        
           tuple ( p(v|h) , v) 
           both are shaped (size of mini-batch, size of visible layer)
        """
        
        assert self.weight_vh is not None

        n_samples = hidden_minibatch.shape[0]

        if self.is_top:

            """
            Here visible layer has both data and labels. Compute total input for each unit (identical for both cases), \ 
            and split into two parts, something like support[:, :-self.n_labels] and support[:, -self.n_labels:]. \
            Then, for both parts, use the appropriate activation function to get probabilities and a sampling method \
            to get activities. The probabilities as well as activities can then be concatenated back into a normal visible layer.
            """

            # [TODO TASK 4.1] compute probabilities and activations (samples from probabilities) of visible layer (replace the pass below). \
            # Note that this section can also be postponed until TASK 4.2, since in this task, stand-alone RBMs do not contain labels in visible layer.
            
            
            pass
            
        else:
                        
            # [DONE TASK 4.1] compute probabilities and activations (samples from probabilities) of visible layer (replace the pass and zeros below)

            probs = sigmoid(hidden_minibatch @ self.weight_vh.T + self.bias_v)
            binary_states = np.random.binomial(1, probs, size=None)             

            pass
        
        return probs, binary_states


    
    """ rbm as a belief layer : the functions below do not have to be changed until running a deep belief net """

    

    def untwine_weights(self):
        
        self.weight_v_to_h = np.copy( self.weight_vh )
        self.weight_h_to_v = np.copy( np.transpose(self.weight_vh) )
        self.weight_vh = None

    def get_h_given_v_dir(self,visible_minibatch):

        """Compute probabilities p(h|v) and activations h ~ p(h|v)

        Uses directed weight "weight_v_to_h" and bias "bias_h"
        
        Args: 
           visible_minibatch: shape is (size of mini-batch, size of visible layer)
        Returns:        
           tuple ( p(h|v) , h) 
           both are shaped (size of mini-batch, size of hidden layer)
        """
        
        assert self.weight_v_to_h is not None

        n_samples = visible_minibatch.shape[0]

        # [TODO TASK 4.2] perform same computation as the function 'get_h_given_v' but with directed connections (replace the zeros below) 
        
        return np.zeros((n_samples,self.ndim_hidden)), np.zeros((n_samples,self.ndim_hidden))


    def get_v_given_h_dir(self,hidden_minibatch):


        """Compute probabilities p(v|h) and activations v ~ p(v|h)

        Uses directed weight "weight_h_to_v" and bias "bias_v"
        
        Args: 
           hidden_minibatch: shape is (size of mini-batch, size of hidden layer)
        Returns:        
           tuple ( p(v|h) , v) 
           both are shaped (size of mini-batch, size of visible layer)
        """
        
        assert self.weight_h_to_v is not None
        
        n_samples = hidden_minibatch.shape[0]
        
        if self.is_top:

            """
            Here visible layer has both data and labels. Compute total input for each unit (identical for both cases), \ 
            and split into two parts, something like support[:, :-self.n_labels] and support[:, -self.n_labels:]. \
            Then, for both parts, use the appropriate activation function to get probabilities and a sampling method \
            to get activities. The probabilities as well as activities can then be concatenated back into a normal visible layer.
            """
            
            # [TODO TASK 4.2] Note that even though this function performs same computation as 'get_v_given_h' but with directed connections,
            # this case should never be executed : when the RBM is a part of a DBN and is at the top, it will have not have directed connections.
            # Appropriate code here is to raise an error (replace pass below)
            
            pass
            
        else:
                        
            # [TODO TASK 4.2] performs same computaton as the function 'get_v_given_h' but with directed connections (replace the pass and zeros below)             

            pass
            
        return np.zeros((n_samples,self.ndim_visible)), np.zeros((n_samples,self.ndim_visible))        
        
    def update_generate_params(self,inps,trgs,preds):
        
        """Update generative weight "weight_h_to_v" and bias "bias_v"
        
        Args:
           inps: activities or probabilities of input unit
           trgs: activities or probabilities of output unit (target)
           preds: activities or probabilities of output unit (prediction)
           all args have shape (size of mini-batch, size of respective layer)
        """

        # [TODO TASK 4.3] find the gradients from the arguments (replace the 0s below) and update the weight and bias parameters.
        
        self.delta_weight_h_to_v += 0
        self.delta_bias_v += 0
        
        self.weight_h_to_v += self.delta_weight_h_to_v
        self.bias_v += self.delta_bias_v 
        
        return
    
    def update_recognize_params(self,inps,trgs,preds):
        
        """Update recognition weight "weight_v_to_h" and bias "bias_h"
        
        Args:
           inps: activities or probabilities of input unit
           trgs: activities or probabilities of output unit (target)
           preds: activities or probabilities of output unit (prediction)
           all args have shape (size of mini-batch, size of respective layer)
        """

        # [TODO TASK 4.3] find the gradients from the arguments (replace the 0s below) and update the weight and bias parameters.

        self.delta_weight_v_to_h += 0
        self.delta_bias_h += 0

        self.weight_v_to_h += self.delta_weight_v_to_h
        self.bias_h += self.delta_bias_h
        
        return    


In [33]:
image_size = [28,28]
train_imgs,train_lbls,test_imgs,test_lbls = read_mnist(dim=image_size, n_train=60000, n_test=10000)

''' restricted boltzmann machine '''

print ("\nStarting a Restricted Boltzmann Machine..")

rbm = RestrictedBoltzmannMachine(ndim_visible=image_size[0]*image_size[1],
                                    ndim_hidden=200,
                                    is_bottom=True,
                                    image_size=image_size,
                                    is_top=False,
                                    n_labels=10,
                                    batch_size=1
)




Starting a Restricted Boltzmann Machine..


In [34]:
visible_trainset = train_imgs

In [36]:
n_samples = visible_trainset.shape[0]
index = 0 

In [37]:
next_index = index + rbm.batch_size


In [38]:
if next_index < n_samples:
    v_0 = visible_trainset[index:next_index]
else:
    v_0 = np.concatenate((visible_trainset[index:],visible_trainset[:next_index-n_samples]))
    print(index)
index = next_index % n_samples

In [39]:
h_0_prob, h_0_bin = rbm.get_h_given_v(v_0)
h_0_bin, h_0_prob

(array([[1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
         0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
         0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0,
         1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1,
         0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
         0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
         0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
         0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1,
         0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
         0, 1]]),
 array([[0.42375982, 0.64900002, 0.41266226, 0.66588423, 0.62112613,
         0.63833397, 0.41005009, 0.66247842, 0.41006205, 0.65846291,
         0.64859548, 0.43232736, 0.63624507, 0.64916533, 0.44694191,
         0.66556608, 0.42728521, 0.63659416, 0.66363595, 0.62636174,
         0.63191945, 0.6657586 

In [40]:
v_1_prob, v_1_bin = rbm.get_v_given_h(h_0_prob)
v_1_prob, v_1_bin

(array([[0.42140182, 0.42899015, 0.42564638, 0.44398823, 0.44669331,
         0.40661191, 0.45251741, 0.45547469, 0.44351111, 0.45525818,
         0.42286012, 0.42253531, 0.4575813 , 0.42862119, 0.40701121,
         0.45170332, 0.44968311, 0.41937595, 0.42773317, 0.44844417,
         0.44930286, 0.44844064, 0.44112257, 0.42627967, 0.4213384 ,
         0.41782192, 0.43200656, 0.43453885, 0.42779708, 0.43479559,
         0.44051703, 0.44357403, 0.45877859, 0.42386065, 0.4289647 ,
         0.41255279, 0.39793455, 0.42995658, 0.40446651, 0.47949412,
         0.4340866 , 0.46025246, 0.4633722 , 0.4042163 , 0.41924759,
         0.39892437, 0.44083864, 0.40945827, 0.4323377 , 0.4058061 ,
         0.43413873, 0.41822923, 0.44871141, 0.43476685, 0.43772876,
         0.40874741, 0.44006944, 0.42519373, 0.44022164, 0.42368145,
         0.42920335, 0.41827244, 0.41576927, 0.39025489, 0.40699716,
         0.43001328, 0.41136934, 0.45383459, 0.42346628, 0.43360883,
         0.43644805, 0.44732804, 0

In [41]:
h_1_prob, h_1_bin = rbm.get_h_given_v(v_1_bin)
h_1_prob, h_1_bin

(array([[0.30155607, 0.40950653, 0.23923101, 0.4114678 , 0.40579124,
         0.45078854, 0.24904557, 0.37202085, 0.28105247, 0.39926623,
         0.43167611, 0.31739041, 0.39277636, 0.35833381, 0.31885275,
         0.46923783, 0.26825435, 0.45423854, 0.43395257, 0.36481621,
         0.46663084, 0.41336159, 0.31898821, 0.46061837, 0.38671542,
         0.40512551, 0.27564614, 0.35877992, 0.47737553, 0.37663021,
         0.26272877, 0.27354466, 0.24458893, 0.48127048, 0.39993492,
         0.47245357, 0.47187091, 0.46513932, 0.38928976, 0.22950789,
         0.41718493, 0.29052067, 0.37546832, 0.37580324, 0.35032997,
         0.32710236, 0.31965284, 0.28378791, 0.39511843, 0.53232844,
         0.44166113, 0.27120126, 0.41518702, 0.43400946, 0.24596732,
         0.28753851, 0.48116252, 0.43202324, 0.44804412, 0.24833096,
         0.37837707, 0.22444136, 0.30844939, 0.51341208, 0.2602912 ,
         0.3759609 , 0.36731032, 0.2348389 , 0.45930542, 0.43492936,
         0.47081139, 0.41186881, 0

In [42]:
rbm.update_params(v_0, h_0_bin, v_1_prob, h_1_prob)

In [45]:
rbm.weight_vh

array([[-0.00087062, -0.01935744, -0.01793766, ...,  0.00667793,
        -0.01252888,  0.01143045],
       [-0.01256875, -0.00406537, -0.00106786, ..., -0.00918752,
        -0.01540138, -0.01647382],
       [ 0.01666596, -0.00778327, -0.00301419, ..., -0.00499257,
        -0.02001859, -0.01423958],
       ...,
       [ 0.01158802,  0.00778681,  0.0045401 , ..., -0.00092714,
        -0.01814085,  0.00241645],
       [ 0.00128996, -0.00919223, -0.01783769, ..., -0.00508061,
        -0.01508978, -0.01934256],
       [-0.00186642,  0.00010269,  0.00043879, ...,  0.00496276,
        -0.00461471, -0.01877712]])

In [47]:
v_1_bin

array([[1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1,
        0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1,
        0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0,
        1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1,
        0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1,
        0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1,
        1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0,
        1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 

In [48]:
if (it % rbm.rf["period"] == 0 or it ==  )and rbm.is_bottom:
    viz_rf(weights=rbm.weight_vh[:,rbm.rf["ids"]].reshape((rbm.image_size[0],rbm.image_size[1],-1)), it=it, grid=rbm.rf["grid"])

# print progress

if it % rbm.print_period == 0:
    print ("iteration=%7d recon_loss=%4.4f"%(it, np.linalg.norm(v_1_bin - visible_trainset)))

iteration=      0 recon_loss=4672.0340
